## 1. Introduction
BioCRNpyler is a software tool designed to rapidly compile large biological chemical reaction networks (CRNs) from simple user specifications (written in python). It has built in support of a number of models for transcription, translation, and gene expression regulation using components common in _E. coli_ synthetic biology. This tutorial explains how to get started in the simplest way. We create a SBML model of a gene expression system. A promoter that expresses GFP is modeled with appropriate transcription and translation mechanisms. 

At the user level, BioCRNplyer provides a CRNLab class that can be used to create a reaction mixture system with - 
* extract, a buffer, and other desired "extracts"
* DNA components
* specify volume of each extract and concentrations of DNA components

After adding all components, a combined model can be obtained as an SBML model object that can be simulated using your simulator of choice. 


## Key Features: 
* Write your own custom "extract" in `extracts.py` and simply use them with `txtl.mixture` in the `extract` argument.
* Specify your own parameter file in the `mixture_parameters` argument (look at `Perfect Param File Example.tsv` for a specimen) or just use default files available with bioCRNpyler.
* Specify the volume of a extract being added in `txtl.mixture` using mixture_volume keyword argument or just specify the final_volume for the model.
* Control warning messages with arguments for `parameter_warning` and `warning` for general warnings (as a boolean).


### For more details on how BioCRNpyler works, refer to the Developer Overview, and Advanced Examples ipython notebooks.

In [2]:
%matplotlib inline

from biocrnpyler import *
import numpy as np

txtl = CRNLab("GFP")


txtl.mixture("mixture1", extract = "BasicExtract", mixture_volume = 1e-6, mixture_parameters = 'BasicExtract.tsv')

# Or simply add a new dna in one line as follows.
txtl.add_dna(name = "G1", promoter = "pBest", rbs = "BCD2", protein = "GFP", initial_conc = 10, volume = 1e-7)

# Combine all of the tubes together to get the model
well1 = txtl.get_model()
print(well1)
# Create an SBML file containing the model
filename = "geneexpr.xml"
txtl.write_sbml_file(filename)
txtl.validate_sbml_generated()
print('SBML model successfully written to {0}'.format(filename))

# Run a simulation (using bioscrape or your own simulator) and plot the result
# (Optional) Specify the type of simulation (deterministic or stochastic)
# timepoints = np.linspace(0,14*60,100)
# simdata = well1.runsim_bioscrape(timepoints, filename, simtype = "stochastic", plot_show = True)

Species = protein_RNAP, protein_Ribo, protein_RNAase, complex_rna_G1_protein_Ribo, protein_GFP, dna_G1, rna_G1, complex_rna_G1_protein_RNAase, complex_dna_G1_protein_RNAP
Reactions = [
	dna_G1 + protein_RNAP <--> complex_dna_G1_protein_RNAP        massaction: k_f=100.0,	k_r=10.0
	complex_dna_G1_protein_RNAP --> dna_G1 + rna_G1 + protein_RNAP        massaction: k_f=3.0
	rna_G1 + protein_Ribo <--> complex_rna_G1_protein_Ribo        massaction: k_f=10.0,	k_r=0.25
	complex_rna_G1_protein_Ribo --> rna_G1 + protein_GFP + protein_Ribo        massaction: k_f=2.0
	rna_G1 + protein_RNAase <--> complex_rna_G1_protein_RNAase        massaction: k_f=10.0,	k_r=0.5
	complex_rna_G1_protein_RNAase --> protein_RNAase        massaction: k_f=1.0
]
The SBML model generated is a valid document according to the SBML Level 3 Version 1 specifications. Use sbml.org/validator for further troubleshooting.
SBML model successfully written to geneexpr.xml


### You can create your own custom mechanisms, components, or mixtures. Refer to the advanced "Developer Overview" notebook for more details

In [3]:
from bokeh.models import (Plot , Range1d)
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()
DG, DGspec, DGrxn = well1.generate_networkx_graph()
plot = Plot(plot_width=500, plot_height=500, x_range=Range1d(-500, 500), y_range=Range1d(-500, 500))
graphPlot(DG,DGspec,DGrxn,plot)
bokeh.io.show(plot)

Loading BokehJS ...

C:\Users\andrey\Anaconda3\lib\site-packages\bokeh\models\graphs.py:164: UserWarning: Node keys in 'layout_function' don't match node keys in the graph. These nodes may not be displayed correctly.
  warn("Node keys in 'layout_function' don't match node keys in the graph. "
